In [13]:
import align.detect_face
import facenet
import cv2
import imutils
import numpy as np
import tensorflow as tf

In [3]:
# Constants
minsize = 20
threshold = [0.6, 0.7, 0.7]
factor = 0.709
margin = 44
input_image_size = 160

In [19]:
print('Creating networks and loading parameters')
with tf.Graph().as_default():
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=1)
    sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options, log_device_placement=False))
    with sess.as_default():
        pnet, rnet, onet = align.detect_face.create_mtcnn(sess, None)
sess = tf.Session()
with sess.as_default():
    pnet, rnet, onet = align.detect_face.create_mtcnn(sess, "align")
facenet.load_model("../../../models/facenet/models/20180408-102900/20180408-102900.pb")

Creating networks and loading parameters


ValueError: Variable pnet/conv1/weights already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "/home/damvantai/Documents/projects/pose_human_with_face_recognition/src/align/detect_face.py", line 125, in make_var
    return tf.get_variable(name, shape, trainable=self.trainable)
  File "/home/damvantai/Documents/projects/pose_human_with_face_recognition/src/align/detect_face.py", line 154, in conv
    kernel = self.make_var('weights', shape=[k_h, k_w, c_i // group, c_o])
  File "/home/damvantai/Documents/projects/pose_human_with_face_recognition/src/align/detect_face.py", line 51, in layer_decorated
    layer_output = op(self, layer_input, *args, **kwargs)
